In [ ]:
# to load data
import pandas as pd
import numpy as np

# data visualisation
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import cufflinks as cf
cf.go_offline()
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.figure_factory as ff
import plotly.express as px

# to scale data and possible reduce dimensionality of data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# for data modelling 
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.decomposition import PCA

pd.options.display.max_rows = 1000

In [ ]:
df = pd.read_csv('application_data.csv')

In [ ]:
100* df.isnull().sum()/len(df)

In [ ]:
percent_missing = pd.DataFrame(100* df.isnull().sum()/len(df)).rename(columns={0:'%_Missing'})

In [ ]:
percent_missing[percent_missing['%_Missing']>10].count()
# i will remove any columns with missing values > 10% to simplify the cleaning process

In [ ]:
# i will attempt to get the names of the 49 columns to remove from the dataset
drop_cols = percent_missing[percent_missing['%_Missing']>10].T.columns.tolist()

# then drop them from the dataset
for cols in drop_cols:
    df.drop([cols],axis=1,inplace=True)

In [ ]:
100* df.isnull().sum()/len(df)
# i will remove any records with null values since missing values are less than 1%

In [ ]:
b = pd.DataFrame(100* df.isnull().sum()/len(df)).rename(columns={0:'%_NA'})

In [ ]:
drop_na = b[b['%_NA']>0].T.columns.tolist()

In [ ]:
df = df.dropna(subset=drop_na)

In [ ]:
fig = px.box(df,x='TARGET',y='AMT_CREDIT',title='Box Plot of credit amount')

In [ ]:
fig.show()
# IQR is tighter for those with payment difficulties 
# ($284.4k ~ $737.5k v.s $270k ~ 810k)
# interesting that max loan amount is similar at ~$4mil
# logic might be that banks are only willing loan smaller amounts due to risk of defaulting

In [ ]:
# annuity in this context is the amount the applicant has to repay on a regular basis to repay the loan
fig = px.box(df,x='TARGET',y='AMT_ANNUITY',title='Box Plot of annuity amount')

In [ ]:
fig.show()
# the graph shows that those with troubles repaying are forced into repaying more 
# each time in an attempt by the back to recoup on risky loans

In [ ]:
sns.catplot(x="NAME_CONTRACT_TYPE", col="TARGET", data=df, kind="count", height=4, aspect=.7)
# more cash loans are giving than revolving loans, loans applicants do not have anything valuable to act as a collateral?
sns.catplot(x="CODE_GENDER", col="TARGET", data=df, kind="count", height=4, aspect=.7)
# more loans more given to females than males

In [ ]:
sns.catplot(x="FLAG_OWN_REALTY", col="TARGET", data=df, kind="count", height=4, aspect=.7)
# more 
sns.catplot(x="FLAG_OWN_CAR", col="TARGET", data=df, kind="count", height=4, aspect=.7)
# more 

In [ ]:
sns.catplot(x="REG_CITY_NOT_LIVE_CITY", col="TARGET", data=df, kind="count", height=4, aspect=.7)
sns.catplot(x="REG_CITY_NOT_WORK_CITY", col="TARGET", data=df, kind="count", height=4, aspect=.7)
# seems like a larger portion of defaulters do not live/work in the same city as where they applied for their loans

In [ ]:
sns.catplot(x="NAME_HOUSING_TYPE", col="TARGET", data=df, kind="count", height=5, aspect=1.9)
# a rare rule might be that if someone lives in a rented, municipal, office or co-op apartment
# they are very unlikely to default on payments

In [ ]:
sns.catplot(x="NAME_EDUCATION_TYPE", col="TARGET", data=df, kind="count", height=5, aspect=1.9)
# seems like those with/undergoing higher education + degrees rarely take loans and rarely default

In [ ]:
sns.histplot(x='AMT_INCOME_TOTAL', hue='TARGET',data=df, bins=50, log_scale=True,color='Blue',binwidth=.1)
# there seems to be an extreme record for income, why would someone with so much income need a loan?

In [ ]:
df.loc[df['AMT_INCOME_TOTAL']==df['AMT_INCOME_TOTAL'].max()]
# $117,000,000! it might be a typo, best to remove it
df = df[df['AMT_INCOME_TOTAL'] != 117000000.0]

In [ ]:
df = df.drop(['SK_ID_CURR'],axis=1)

In [ ]:
# cleaning of binary variables with one-hot encoding

df['NAME_CONTRACT_TYPE'] = df['NAME_CONTRACT_TYPE'].replace({'Cash loans':1,'Revolving loans':0})

df['CODE_GENDER'] = df['CODE_GENDER'].replace({'XNA':'F'})
df['CODE_GENDER'] = df['CODE_GENDER'].replace({'F':'1','M':0})

df['FLAG_OWN_CAR'] = df['FLAG_OWN_CAR'].replace({'Y':1,'N':0})

df['FLAG_OWN_REALTY'] = df['FLAG_OWN_REALTY'].replace({'Y':1,'N':0}) 

In [ ]:
# cleaning of the age and days employed and converting it into years and removing negative sign
df['AGE'] = df['DAYS_BIRTH']/-365
df['EMP_LENGTH'] = df['DAYS_EMPLOYED']/-365
df.drop(['DAYS_BIRTH','DAYS_EMPLOYED'],axis=1,inplace=True)

In [ ]:
# list of columns to be used for Kmeans modelling 
col = [
    # numerical variables
    'TARGET', 'CODE_GENDER', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
    'REGION_POPULATION_RELATIVE', 'AGE', 'EMP_LENGTH', 'REGION_RATING_CLIENT', 'EXT_SOURCE_2',
    # categorical variables
    'NAME_CONTRACT_TYPE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE'
]
df_model = df[col]

In [ ]:
# creating a heat map for feature engineering
model_corr = df_model.corr()
mask = np.triu(np.ones_like(model_corr, dtype=np.bool))
# Set up the matplotlib figure
fig, ax = plt.subplots(figsize = (12, 12))
# Draw the heatmap with 'sns.heatmap()'
ax= sns.heatmap(model_corr, mask=mask, annot=True, square=True, linewidth=0.5, vmin=-1, vmax=1, cmap='coolwarm',fmt = '.2f')

In [ ]:
# new variables to reduce dimensionality .

# NUM_PAYMENT = ( [ANNUITY] / [CREDIT] )
# the greater the ratio, the greater the chance to default due to perceived diffculty 
# in recouping principle amount from applicant
df_model['ANNUITY_TO_CREDIT_RATIO'] = df_model['AMT_ANNUITY'] / df_model['AMT_CREDIT']
df_model.drop(['AMT_CREDIT','AMT_ANNUITY'],axis=1,inplace=True)

# NUM_PAYMENT = ( [EMP_LENGTH] / [AGE] )
# the greater the ratio, the lower the chance to default due to a steady source of income for repayment
# in recouping principle amount from applicant
df_model['WORK_TO_AGE_RATIO'] = df_model['EMP_LENGTH'] / df_model['AGE']
df_model.drop(['EMP_LENGTH','AGE'],axis=1,inplace=True)

# RATING_TO_DENSITY_RATIO = ( [REGION_RATING_CLIENT] / [REGION_POPULATION_RELATIVE] )
# the greater the REGION_RATING_CLIENT value, the more likely the applicant will can finanical diffculties with repayment
# this infers that a lower regional rating score is better
# the more populated the region the applicant lives in, the lower their regional rating score (desirable)
# thus, the lower the ratio, the more desirabke it is to loan to the applicant
df_model['RATING_TO_DENSITY_RATIO'] = df_model['REGION_RATING_CLIENT'] / df_model['REGION_POPULATION_RELATIVE']
df_model.drop(['REGION_RATING_CLIENT','REGION_POPULATION_RELATIVE'],axis=1,inplace=True)

In [ ]:
# dropping all categorical variables
df_model_num = df_model.drop(['NAME_CONTRACT_TYPE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE'],axis=1)

In [ ]:
X = df_model_num.drop(['TARGET'], axis=1)
y = df_model_num['TARGET']

In [ ]:
kmeans = KMeans(n_clusters=2)

In [ ]:
kmeans.fit(X)

In [ ]:
print(confusion_matrix(y,kmeans.labels_))
print()
print(classification_report(y,kmeans.labels_))

# Output

# [[237313  42551]
#  [ 21678   2988]]

#              precision    recall  f1-score   support
#           0       0.92      0.85      0.88    279864
#           1       0.07      0.12      0.09     24666

#    accuracy                           0.79    304530
#   macro avg       0.49      0.48      0.48    304530
#weighted avg       0.85      0.79      0.82    304530

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
pca = PCA(random_state=101,n_components=7)

In [ ]:
pca.fit(X_scaled)

In [ ]:
df_comp = pd.DataFrame(pca.components_,columns=X.columns)

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(df_comp,cmap='coolwarm',annot=True)
# i will be ignoring any components with coefficient < 0.00 as it will be insignificant in interepting PCs

In [ ]:
# PC1 : Perfect negative correlation with gender

# PC2 and PC3 seems similar and contradicts

# PC2 : Contrast btw the ext source 2 and relative high annuity:credit ratio
# PC3 : Weight sum btw ext source 2 and relative higher annuity:credit ratio
# PC4 : Perfect negative correlation with work:age ratio
# PC5 : Perfect correlation with # of children
# PC6 : Perfect correlation with rating:density ratio
# PC7 : Perfect correlation with total income

In [ ]:
pca.components_.round(2)

# coefficents for each PC

#array([[-1.  ,  0.  ,  0.  , -0.01,  0.04,  0.05, -0.  ],
#       [ 0.03,  0.  , -0.  , -0.84,  0.55, -0.01,  0.02],
#       [ 0.03,  0.  ,  0.  ,  0.55,  0.84,  0.03, -0.01],
#       [-0.05, -0.07, -0.  ,  0.02,  0.02, -1.  , -0.  ],
#       [ 0.  ,  1.  , -0.  ,  0.01, -0.  , -0.07,  0.06],
#       [-0.  , -0.06, -0.  ,  0.02, -0.  ,  0.  ,  1.  ],
#       [ 0.  ,  0.  ,  1.  , -0.  ,  0.  , -0.  ,  0.01]])

In [ ]:
pca.explained_variance_ratio_.round(2)
# 94% of variance explained with the first 3 PCs

In [ ]:
pca = PCA(random_state=101,n_components = 3)
df_pca = pca.fit_transform(X_scaled)

In [ ]:
y = y.reset_index().drop(['index'],axis=1)
df_pca_model = pd.DataFrame(df_pca, columns=["PCA1", "PCA2", "PCA3"])
#df_pca_model['TARGET'] = y
#df_pca_model = df_pca_model.drop(['PCA4','PCA5'],axis=1)
df_pca_model.head() 

In [ ]:
kmeans = KMeans(random_state=101,n_clusters=2)

In [ ]:
kmeans.fit(df_pca_model) #.drop('TARGET',axis=1)

In [ ]:
kmeans.cluster_centers_

#array([[ 0.65932912, -0.00461518, -0.00436472],
#       [-0.34216973,  0.00239512,  0.00226514]])

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y,kmeans.labels_))
print(classification_report(y,kmeans.labels_))

# Output
#[[ 93441 186423]
# [ 10604  14062]]

#              precision    recall  f1-score   support
#
#           0       0.90      0.33      0.49    279864
#           1       0.07      0.57      0.12     24666

#    accuracy                           0.35    304530
#   macro avg       0.48      0.45      0.31    304530
#weighted avg       0.83      0.35      0.46    304530

In [ ]:
df_pca_model['PRED_TARGET'] = kmeans.labels_
df_pca_model['TARGET'] = y
df_pca_model

In [ ]:
# the new model has made better attempts to correctly classify those with payment diffculties
# but has classified more false positives